In [1]:
import pandas as pd
from typing import List, Dict, Any

In [2]:
url_restauration = "https://www.pxweb.bfs.admin.ch/sq/3c431fe1-16b0-4572-a8cd-ddbcfc919c5f"
url_population = "https://www.pxweb.bfs.admin.ch/sq/a2f3f03c-3b76-4dc0-890e-c6534d341848"

def restauration_employement_to_population_ratio(url_restauration : str, url_population : str) -> pd.DataFrame:
    df_restauration = pd.read_csv(url_restauration, encoding="ANSI")
    df_population = pd.read_csv(url_population, encoding="ANSI")
    df_population = df_population.groupby(["Année", "Sexe"], as_index=False)["Effectif au 1er janvier"].agg("sum")
    
    years : List[int] = [i for i in range(2010, 2025) for _ in range(8)]
    quarters : List[int] = [i for _ in range(15*2) for i in range(1, 5)]
    genders : List[int] = [i for _ in range(15) for i in [1]*4+[0]*4]  # 1 if female, 0 if male
    indexes_of_columns_to_go : List[int] = [2+i for j in range((df_restauration.shape[1]-3)//4) for _ in range(2) for i in range(4*j+1, 4*j+5)] #to search efficiently in the dataframe for having the final datafram in the right format easily
    y : List[float] = []

    for index, gender, year in zip(indexes_of_columns_to_go, genders, years):
        y.append(df_restauration.iloc[gender, index]/df_population.loc[df_population["Année"].eq(year)&df_population["Sexe"].eq("Femme" if gender==1 else "Homme"), "Effectif au 1er janvier"].iat[0]*100)

    dict : Dict[List[Any]] = {"Year" : years,
            "Quarters" : quarters,
            "Employment-to-population Ratio [%]" : y, 
            "Gender" : genders}
    return pd.DataFrame(dict)

In [3]:
df = restauration_employement_to_population_ratio(url_restauration, url_population)
df

,Year,Quarters,Employment-to-population Ratio [%],Gender
0,2010,1,3.103141,1
1,2010,2,3.084915,1
2,2010,3,3.088478,1
3,2010,4,2.990922,1
4,2010,1,2.482187,0
...,...,...,...,...
115,2024,4,2.456345,1
116,2024,1,2.465753,0
117,2024,2,2.494324,0
118,2024,3,2.441515,0
